In [27]:
# Pkg.add("GraphPlot")
using Pkg
# Pkg.add("LightGraphsFlow")
using JuMP, Gurobi
using LightGraphs, LightGraphsFlows
using GraphPlot
const gurobi_env = Gurobi.Env()
function plotGraph(edge)
    global N, origin, destination
#     n = maximum(edge)
#     println(n)
    g = SimpleDiGraph(destination)
    label1 = collect(1:destination)
    for i = 1:length(edge[:,1])
        add_edge!(g, edge[i,1], edge[i,2]);
    end
    gplot(g, layout=circular_layout, nodelabel=label1)
#     kruskal_mst(g, ones(length(edge[:,1])))
#     println(is_strongly_connected(g))
end
function gx_bound(c, c_g, edge, source_sink)
    global  N, origin, destination
#     println("inside")
#     println("\tc = ", c)
#     println("\tc_g = ", c_g)
#     println("\tedge = ", edge)
#     println("\tsource_sink = ", source_sink)
    #println(f,"Current CELL's LB = ", c_L)
    #println(f,"Current CELL's UB = ", c_U)
    #println(f, "Current interdiction x = ", x_now)
    if source_sink == true
        start_node = edge[:,1]
        end_node = edge[:,2]
        origin = 1
        destination = N
    else 
        start_node = edge[:,2]
        end_node = edge[:,1]
        origin = N
        destination = 1
    end
    no_node = max(maximum(start_node), maximum(end_node) )
    no_link = length(start_node)

    function getShortestX(state, start_node, end_node, origin, destination)
        _x = zeros(Int, length(start_node))
        _path = enumerate_paths(state, destination)

        for i=1:length(_path)-1
            _start = _path[i]
            _end = _path[i+1]

            for j=1:length(start_node)
                if start_node[j]==_start && end_node[j]==_end
                _x[j] = 1
                break
                end
            end

        end
        _x
    end


    graph = SimpleDiGraph(no_node)
    distmx = Inf*ones(no_node, no_node)
    
    # Adding links to the graph
    for i=1:no_link
        add_edge!(graph, start_node[i], end_node[i])
        distmx[start_node[i], end_node[i]] = c_g[i]
    end
    
    # Run Dijkstra's Algorithm from the origin node to all nodes
    state = dijkstra_shortest_paths(graph, origin, distmx)
    label = state.dists
    pred = state.parents
    b_arc = ""
    
    for i = 1: length(state.parents)
        if state.parents[i] != 0 
            b_arc = string(b_arc, "(", state.parents[i], ",", i, ")")
        end
    end
    
    # Retrieving the shortest path
    path = enumerate_paths(state, destination)
    
#     parents = LightGraphs.DijkstraState(state, destination)

    # Retrieving the 'x' variable in a 0-1 vector
    y = getShortestX(state, start_node, end_node, origin, destination)
    
#     println("y vector:", y)
#     println("pred = ", pred)
    gx = sum(c_g[i]*y[i] for i = 1:no_link)    
    SP = sum(c[i]*y[i] for i = 1:length(c))
#     println("gx ", gx)
#     println("SP =", SP)
    T = Int64[]
    numArcs = length(edge[:,1])
    for i = 1:numArcs
        
        if pred[edge[i,2]] == edge[i,1]
            push!(T, 1)
        else
            push!(T, 0)
        end
    end
#     println("T = ", T)
    origin = 1
    destination = N
#     println("Minimum Spanning Tree: ", T)
#     println("Shortest path y = ", y)
    #println("Indices of shortest path (edge) = ", y_index)
    #println("Nodes visited =", path)
#     println("Minimum Spanning Tree =", pred)
#     println("Node label = " ,label)
#     println("inside label = ", label)
    return y, gx, SP, label
end

function x_disconnect(arcs, b_x, d_x, origin, destination, y_init)

    numArcs = length(arcs[:,1])
    capacity_matrix = zeros(Int, numArcs, numArcs)
    no_node = maximum(arcs)
    no_link = length(arcs[:,1])
    start_node = arcs[:,1]
    end_node = arcs[:,2]

    graph = SimpleDiGraph(no_node)
    distmx = Inf*ones(no_node, no_node)

    # Adding links to the graph
    for i=1:no_link
        add_edge!(graph, start_node[i], end_node[i])
    #     distmx[start_node[i], end_node[i]] = c_g[i]
        capacity_matrix[start_node[i], end_node[i]] = d_x[i]
    end
    f, F = maximum_flow(graph, origin, destination, capacity_matrix)
    
    if f <= b_x
        is_connected = false
    else
        is_connected = true
    end
    return is_connected,f
end

# function print2File()

# edge = [1 4; 2 6; 2 8; 2 10; 2 11; 2 15; 3 11; 4 13; 5 3; 5 13; 6 8; 7 2; 7 11; 7 15; 8 4; 8 7; 8 12; 8 15; 9 2; 9 6; 9 8; 9 13; 10 11; 11 10; 12 2; 12 5; 12 6; 12 11; 13 2; 13 7; 13 14; 14 5; 14 9; 14 13]
# edge = [1 2; 1 3; 3 2]
# global Len = length(edge[:,1])
# c = ones(Len)
# c_g = ones(Len)
# global origin = 1
# global destination = 3

Academic license - for non-commercial use only - expires 2022-12-04


x_disconnect (generic function with 1 method)

In [28]:
ins = 5#ARGS[1] #:10
# dataSet = "N20_d40_"
# include("./"*dataSet*string(ins)*".jl")
include("./test.jl")
N = destination
numArcs =length(arcs[:,1])
Len = numArcs
c = ones(numArcs)
println(arcs)
y_init, gx, SP, label1 = gx_bound(c, c, arcs, true)
println("gx = ", gx)
println("SP = ", SP)
println("1")
is_connected = x_disconnect(arcs, b_x, d_x, origin, destination, y_init)
println(is_connected)
println("budget = ", b_x)
# println(arcs)
# println(d_y)
# mA= [16,17,18, 5, 19,8]
# println(arcs[mA,:])
# println(d_y[mA])
# plotGraph(arcs)

#[3 7; 5 10]
# y, gx, SP = gx_bound(c, c_g, edge)

[1 4; 1 6; 1 9; 1 13; 1 14; 1 17; 1 19; 2 6; 2 8; 2 9; 2 12; 2 15; 3 5; 3 8; 3 9; 3 10; 3 11; 3 15; 3 16; 3 18; 4 2; 4 5; 4 6; 4 8; 4 9; 4 11; 4 12; 4 15; 4 16; 4 17; 4 18; 4 19; 5 10; 5 11; 5 12; 5 13; 5 15; 5 17; 5 20; 6 3; 6 4; 6 7; 6 13; 6 15; 6 16; 6 18; 6 20; 7 2; 7 4; 7 5; 7 6; 7 13; 7 14; 7 15; 7 16; 7 19; 8 2; 8 3; 8 4; 8 5; 8 6; 8 7; 8 9; 8 12; 8 13; 8 14; 8 16; 8 18; 8 19; 8 20; 9 2; 9 6; 9 8; 9 13; 9 14; 10 5; 10 6; 10 7; 10 12; 10 14; 10 15; 10 16; 10 19; 11 12; 11 15; 12 2; 12 4; 12 6; 12 7; 12 14; 12 15; 12 19; 13 5; 13 6; 13 7; 13 8; 13 10; 13 14; 13 15; 13 16; 13 18; 13 19; 14 3; 14 7; 14 9; 14 10; 14 17; 14 18; 14 20; 15 2; 15 5; 15 6; 15 9; 15 10; 15 11; 15 16; 15 19; 16 2; 16 7; 16 10; 16 11; 16 13; 16 17; 16 18; 16 19; 17 2; 17 5; 17 8; 17 9; 17 16; 17 19; 17 20; 18 3; 18 12; 18 15; 18 17; 18 19; 18 20; 19 2; 19 4; 19 8; 19 9; 19 12; 19 13; 19 14; 19 15; 19 17; 19 20]
gx = 2.0
SP = 2.0
1
(true, 38)
budget = 10


In [20]:
println("running....")
# Pkg.add("GraphPlot")
# num = "14"
# edge = Int64[]
global edge = Array{Int64}(undef, 0, 2)
global N = 10
global origin = 1
global destination = N
global numArcs = 0
global b_x = 5
global b_y = 10
global d_x = []
global d_y = []
println("...")
# global uncNum = 0.1 # 0.2
for num = 1:2 #00 #0
    print("",num,"...")
    global edge, N, origin, destination, numArcs, d_x, d_y#, uncNum
    
    stop_criterion = false
    density = 0.1
    times = 0
#     println("before stop_crit")
    while stop_criterion == false
#         times = times+1
#         if times > 5
#             break
#         end
        edge = Array{Int64}(undef, 0, 2)
        head = zeros(N)
        tail = zeros(N)
            for u = 1 : N-1
                for v = 2 : N
                    x = rand()
                    if x <= density && u != v 
                        edge = vcat(edge,[u v])
                    end
                end
            end
#         println("",edge)
        numArcs = length(edge[:,1])
        c_temp = ones(numArcs)
        
        #Verify there is a path from source to every node and from every node to sink
        try
            print("x")
            y_init, gx, SP, label1 = gx_bound(c_temp, c_temp, edge, true)
#             println("y_init = ", y_init)
#             println("label1 = ", label1)
            y_init2, gx, SP, label2 = gx_bound(c_temp, c_temp, edge, false)
#             println("labl2 = ", label2)
            if isfinite(maximum(label1))==true && isfinite(maximum(label2)) == true
#                 stop_criterion = true
#                 println("numArcs = ", numArcs)
                d_x = Float64[]
                d_y = Float64[]
                for i = 1 : numArcs
                    x = rand(1:10)
                    y = rand(1:10)
                    push!(d_x, x)
                    push!(d_y, y)
                end
                stop_criterion,f = x_disconnect(edge, b_x, d_x, origin, destination, y_init)
                println(stop_criterion,". max flow = ", f, " vs b_x = ", b_x)
                
            end
        catch
            stop_criterion = false
        end
        
        
    end
    println(edge)
    println(length(edge[:,1]))
    plotGraph(edge)
    
    #Generate q-values as detection probability in (0,1) , 3 decimals
    q = zeros(numArcs)
    p = rand(0:999,numArcs) #Float64[]
    for i = 1 : numArcs
#         p = 
        q[i] = rand(p[i]:999)#/1000
#         if x<0
#             x=0
#         end
#         push!(q, x)
    end
    q = q./1000
    p = p./1000
    println("p = ", p)
    println("q = ", q)
    println("y")
#     println(cL)
#     println(cU)
    # println(myindex)
    origin = 1
    destination = N
    
#     println("Here:", Len) 
    y, gx, SP, label = gx_bound(q, q, edge, true)
    # println(gx)
#     println("HERE")
    
    outLoc = "C:/Users/din/Documents/GitHub/UncertainTarget/EnumX_RowCol2ndStage/TestInstances/p_Instances/N"*string(N)
    # println(d)
    d_name = convert(Int64, density*100) #int(density*100)
    outfile = outLoc*"/N"*string(N)*"_d"*string(d_name)*"_"*string(num)*".jl"
#     outfile = "/Users/din/Desktop/Research/GenerateTestSet/Test_"*string(num)*".jl"
    f = open(outfile, "w")
    println(outfile)
    println(f,"global arcs = ", edge) #length(df[:CELL]))
    println(f,"global d_x = ", d_x)
    println(f,"global b_x = ", b_x)
    println(f,"global d_y = ", d_y)
    println(f,"global b_y = ", b_y)
    println(f,"global p = ", p)
    println(f,"global q = ", q)
    println(f,"global origin = 1")
    println(f,"global destination = ", N)
    close(f)
end

running....
...
1...xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

In [ ]:
import Statistics
set_ins = rand(1:200,20)
temp_ = rand(20)
println(Statistics.mean(temp_[1:5]))
# average()
 for num in set_ins
#     include("./N50_Ins_"*string(num)*".jl")
#     A = length(edge)
#     println(num, ". min cL", minimum(cL_orig), "; max M ", maximum(M_orig[findall(M_orig.<20)]), "; max d ", maximum(d))
#     println("density = ", A/destination/(destination-1))
# end

In [ ]:
using LightGraphs, LightGraphsFlows
edge = [1 2; 2 3; 1 3]
c_g = [1, 1.5, 10]
dataSet = "N10_d10_"
ins = 1
println("Running...")
#     include("./TestInstances/"*fileName*".jl")
# global q = []
include("./"*dataSet*string(ins)*".jl")
edge = arcs
c_g = d_x
numArcs = length(arcs[:,1])
capacity_matrix = zeros(Int, numArcs, numArcs)
# function dijkstra(c_g, edge)
#     no_node = maximum(edge)
#     no_link = length(edge[:,1])
#     start_node = edge[:,1]
#     end_node = edge[:,2]
    
#     graph = SimpleDiGraph(no_node)
#     distmx = Inf*ones(no_node, no_node)

# # Adding links to the graph
#     for i=1:no_link
#         add_edge!(graph, start_node[i], end_node[i])
#         distmx[start_node[i], end_node[i]] = c_g[i]
#     end
#     A = dijkstra_shortest_paths(graph, origin, distmx)
#     return A
# end
# A = dijkstra(c_g, edge)
no_node = maximum(edge)
no_link = length(edge[:,1])
start_node = edge[:,1]
end_node = edge[:,2]

graph = SimpleDiGraph(no_node)
distmx = Inf*ones(no_node, no_node)

# Adding links to the graph
for i=1:no_link
    add_edge!(graph, start_node[i], end_node[i])
#     distmx[start_node[i], end_node[i]] = c_g[i]
    capacity_matrix[start_node[i], end_node[i]] = d_x[i]
end
f, F = maximum_flow(graph, origin, destination, capacity_matrix)
println(f)
println(b_x)
println(F)
plotGraph(arcs)

In [ ]:
#Generate New Files Unc Range /2 and *2 for each arc
# global edge = Array{Int64}(undef, 0, 2)
# global N = 50 #100
# global origin = 1
# global destination = N
# global Len = 0 
function print2File(cL, cU, stat, myFile)
    println(stat)
    println(myFile)
    outfile = "./N50_"*stat*"UncRange_"*myFile*".jl"
    println(outfile)
#     outfile = "/Users/din/Desktop/Research/GenerateTestSet/Test_"*string(num)*".jl"
    f = open(outfile, "w")
    println(f,"edge = ", edge) #length(df[:CELL]))
    println(f,"cL_orig = ", cL)
    println(f,"cU_orig = ", cU)
    println(f,"d = ", d)
    println(f,"Len = length(cL_orig)")

    println(f,"c_orig = 0.5*(cL_orig+cU_orig)")

    #STARTING SOLUTION:
    println(f,"yy = zeros(Len)")
    println(f,"yy = ", yy)
    println(f,"SP_init = sum(yy[i]*c_orig[i] for i = 1:Len)")
    println(f,"p = [1.0]")
    println(f,"g = [SP_init]")
    println(f,"h = [0.0]")

    println(f,"origin = 1")
    println(f,"destination = 50")

    println(f,"last_node = maximum(edge)")
    println(f,"all_nodes = collect(1:last_node)")

    println(f,"M_orig = zeros(Len)")

    println(f,"for i = 1:Len")
    println(f,"    M_orig[i] = cU_orig[i] - cL_orig[i]")
    println(f,"end")


    println(f,"delta1 = ", delta1)
    println(f,"delta2 = ", delta2)
    println(f, "b = ", b) #rand(1:b_max))
    
#     println(f, )
    println(f,"last_node = maximum(edge)")
    println(f,"β = ", β)
    close(f)
end
function divideby2(cL, cU, M, myFile)
    c_bar = 0.5.*(cL + cU)
    M = 0.25.*M
    cL = c_bar - M
    cU = c_bar + M
    print2File(cL, cU, "Half", myFile)
#     println("\ncL = ", cL[M.>0])
#     println("cU = ", cU[M.>0])
end
function multby2(cL, cU, M, myFile)
    c_bar = 0.5.*(cL + cU)
#     M = 0.25.*M
    cL = c_bar - M
    cU = c_bar + M
    A_neg = findall(cL.<0)
#     println("\nA_neg = ", A_neg)
    for a in A_neg
        cU[a] = cU[a]-cL[a]
        cL[a] = 0
    end
    print2File(cL, cU, "Double", myFile)
#     println(cL[A_neg])
#     println("\ncL = ", cL[M.>0])
#     println("cU = ", cU[M.>0])
end

for ins in [16,35,41,43,56,79,84,110,118,142]
    myFile = string(ins)#*"_lastr"
    include("./N50/N50_"*myFile*".jl")
#     println("cL = ", cL_orig)
#     println("M_orig = ", M_orig)
    uncA = findall(M_orig.>0)
    println("density = ", length(uncA)/length(edge[:,1]))
#     println("uncA (", length(uncA),") = ", uncA)
#     println("cL_orig = ", cL_orig[M_orig.>0])
#     println("cU_orig = ", cU_orig[M_orig.>0])
    
    divideby2(cL_orig, cU_orig, M_orig,myFile)
    multby2(cL_orig, cU_orig, M_orig, myFile)
#     println("M_orig = ", M_orig[M_orig.>0])
end

In [ ]:
#Change budget to b=4 and b=10 for each set:
function updateBudgetb(myFile,b_new)
    println("b_new = ", b_new)
    println(myFile)
    outfile = "./N50_b"*string(b_new)*"_"*myFile*".jl"
    println(outfile)
#     outfile = "/Users/din/Desktop/Research/GenerateTestSet/Test_"*string(num)*".jl"
    f = open(outfile, "w")
    println(f,"edge = ", edge) #length(df[:CELL]))
    println(f,"cL_orig = ", cL_orig)
    println(f,"cU_orig = ", cU_orig)
    println(f,"d = ", d)
    println(f,"Len = length(cL_orig)")

    println(f,"c_orig = 0.5*(cL_orig+cU_orig)")

    #STARTING SOLUTION:
    println(f,"yy = zeros(Len)")
    println(f,"yy = ", yy)
    println(f,"SP_init = sum(yy[i]*c_orig[i] for i = 1:Len)")
    println(f,"p = [1.0]")
    println(f,"g = [SP_init]")
    println(f,"h = [0.0]")

    println(f,"origin = ", origin)
    println(f,"destination = ", destination)

    println(f,"last_node = maximum(edge)")
    println(f,"all_nodes = collect(1:last_node)")

    println(f,"M_orig = zeros(Len)")

    println(f,"for i = 1:Len")
    println(f,"    M_orig[i] = cU_orig[i] - cL_orig[i]")
    println(f,"end")


    println(f,"delta1 = ", delta1)
    println(f,"delta2 = ", delta2)
    println(f, "b = ", b_new) #rand(1:b_max))
    
#     println(f, )
    println(f,"last_node = maximum(edge)")
    println(f,"β = ", β)
    close(f)
end

for ins in [16,35,41,43,56,79,84,110,118,142]
    myFile = string(ins)#*"_lastr"
    include("./N50/N50_"*myFile*".jl")
#     println("density = ", length(uncA)/length(edge[:,1]))
    b_new = 4
    updateBudgetb(myFile, b_new)
    b_new = 10
    updateBudgetb(myFile, b_new)
end

In [ ]:

set = rand(1:200, 10)
for ins in set # [16,35,41,43,56,79,84,110,118,142]
    myFile = string(ins)#*"_lastr"
    dataSet = "N100"
    include("./"*dataSet*"/"*dataSet*"_"*myFile*".jl")
#     println("cL = ", cL_orig)
#     println("M_orig = ", M_orig)
    uncA = findall(M_orig.>0)
    println(ins," .UncNum = ", length(uncA)/length(edge[:,1]))
    println("\t density = ", length(d)/destination/(destination-1))
    println("\t maxM = ", maximum(M_orig))
    println("\t min c = ", minimum(cL_orig))
    println("\t b = ", b)
end

In [ ]:
rand(1:999,10)

In [11]:
global arcs = [1 2; 1 5; 1 8; 2 5; 2 6; 3 5; 3 9; 4 2; 4 7; 4 8; 4 9; 5 4; 5 6; 5 7; 6 7; 7 2; 7 3; 8 2; 8 6; 9 3; 9 4; 9 10]
global d_x = [6.0, 9.0, 7.0, 7.0, 6.0, 1.0, 10.0, 3.0, 6.0, 8.0, 3.0, 5.0, 6.0, 6.0, 3.0, 2.0, 7.0, 1.0, 1.0, 9.0, 9.0, 9.0]

a_ = findall(d_x.>=5)
println(arcs[a_, :])

[1 2; 1 5; 1 8; 2 5; 2 6; 3 9; 4 7; 4 8; 5 4; 5 6; 5 7; 7 3; 9 3; 9 4; 9 10]


In [10]:
println(d_x[a_])

[1.0, 3.0, 3.0, 5.0, 3.0, 2.0, 1.0, 1.0]


In [26]:
N = 35
d_name = 10

outLoc = "C:/Users/din/Documents/GitHub/UncertainTarget/EnumX_RowCol2ndStage/TestInstances/p_Instances/N"*string(N)

for num = 1:100
    include(outLoc*"/N"*string(N)*"_d"*string(d_name)*"_"*string(num)*".jl")
    a_ = findall(p.==0)
    if length(a_) > 0
        println("ins = ", num)
    end
end

ins = 9
ins = 31
ins = 32
ins = 34
ins = 45
ins = 46
ins = 65
ins = 73
ins = 76
ins = 89
ins = 91
ins = 98
